In [39]:
%%writefile spacex_dash.py

import pandas as pd
from dash import Dash, html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

launch_site_names = spacex_df['Launch Site'].unique()

# Create options for dropdown dynamically
dropdown_options = [{'label': launch_site, 'value': launch_site} for launch_site in launch_site_names]

# Add the 'All Sites' option
dropdown_options.insert(0, {'label': 'All Sites', 'value': 'ALL'})

# Create a dash application
app = Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Div(["Launch Site: ", dcc.Dropdown(id='site-dropdown',
                                                                                        options=dropdown_options,
                                                                                        value='ALL',
                                                                                        placeholder="place holder here",
                                                                                        searchable=True),]),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                html.Div(dcc.RangeSlider(id='payload-slider',
                                                        min=0, max=10000, step=1000,
                                                        marks={0: '0',
                                                        100: '100'},
                                                        value=[min_payload, max_payload]),),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))


def get_pie_chart(entered_site):
    filtered_df = spacex_df.groupby('Launch Site')['class'].sum().reset_index()
    if entered_site == 'ALL':
        fig = px.pie(filtered_df,
                values='class',
                names='Launch Site', 
                title='Total Success Launch by site')
        return fig
    else:
        site_filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        site_fig = px.pie(site_filtered_df,
                          names='class',
                          title=f'Launch Outcomes at {entered_site}')
        return site_fig
        # return the outcomes piechart for a selected site

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'), 
             [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")])

def get_scatter_chart(entered_site, payload_range):
    if entered_site == 'ALL':
        filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload_range[0]) & (spacex_df['Payload Mass (kg)'] <= payload_range[1])]
        fig = px.scatter(filtered_df,
                         x='Payload Mass (kg)',
                         y='class',
                         color='Booster Version Category',
                         title='Correlation between Payload and Launch Success for All Sites')
        return fig
    else:
        site_filtered_df = spacex_df[(spacex_df['Launch Site'] == entered_site) & 
                                     (spacex_df['Payload Mass (kg)'] >= payload_range[0]) & 
                                     (spacex_df['Payload Mass (kg)'] <= payload_range[1])]
        site_fig = px.scatter(site_filtered_df,
                              x='Payload Mass (kg)',
                              y='class',
                              color='Booster Version Category',
                              title=f'Correlation between Payload and Launch Success at {entered_site}')
        return site_fig



# Run the app
if __name__ == '__main__':
    app.run_server()


Overwriting spacex_dash.py
